In [24]:
"""
Module for handling with index database
"""

from redis import Redis
import pandas as pd


redis_conn = Redis(decode_responses=True)


def update_index(docs, stemmed):
    """
    docs - list or array of strs
    stemmed - list of lists with token strs
    returns ids of updated docs
    """

    pipe = redis_conn.pipeline()

    max_id = pipe.get("max doc id").execute()[0]
    if max_id is None:
        max_id = 0
    else:
        max_id = int(max_id)

    new_ids = range(max_id, len(docs), 1)

    for doc, stm_doc, doc_id in zip(docs, stemmed, new_ids):
        #  update index
        #  TODO: make doc great again! (as a HASH)
        pipe.set("doc:{}".format(doc_id), doc)
        #  update inverted index
        for tok in stm_doc:
            pipe.sadd(tok, doc_id)
    pipe.incrby("max doc id", len(docs))
    pipe.execute()


def from_eval_texts(path='../../Data/eval_texts.csv'):
    """
    Method for hydrating index from dump csv data
    """
    pipe = redis_conn.pipeline()

    df = pd.read_csv(path, sep='\t')
    documents = df.text.values[:100]
    documents_stemmed = df.text_searchable.values[:100]
    stemmed = [s_doc.split() for s_doc in documents_stemmed]
    return update_index(documents, stemmed)


def search(tokens):
    """
    tokens - list of strs
    returns set of doc_ids strs
    """
    pipe = redis_conn.pipeline()
    ids = pipe.sinter(*tokens).execute()[0]
    return ids


def get_docs(ids, is_str=False):
    """
    ids - list of ids (integers or strings like "doc:<id>")
    returns list of strs
    """
    pipe = redis_conn.pipeline()
    if is_str:
        for doc_id in ids:
            pipe.get(doc_id)
    else:
        for doc_id in ids:
            pipe.get("doc:{}".format(doc_id))
    return pipe.execute()
    
    
from_eval_texts(path='../../../Data/eval_texts.csv')

In [25]:
pipe = redis_conn.pipeline()
pipe.get("doc:0").execute()
# pipe.get("max doc id").execute()

['Должностные обязанности: * Грамотная выкладка товара Чиртон * Контроль ценников, сроков годности * Размещение рекламных материалов * Предоставление фотоотчета по эл.почте, whatsapp Требования: Опыт работы приветствуется Наличие л/а Предпочтительны кандидаты без опыта-всему с нуля обучаем! Водительские права категории: B. Мы предлагаем: * Официальное трудоустройство * гсм входит в заработную плату Полный рабочий день Разъездного характера']

In [26]:
pipe.sinter('обязанност', 'должност').execute()[0]

{'0',
 '10',
 '12',
 '19',
 '2',
 '23',
 '26',
 '34',
 '35',
 '40',
 '44',
 '46',
 '54',
 '56',
 '73',
 '86',
 '9',
 '90',
 '94'}

In [27]:
get_docs([0,1,2])

['Должностные обязанности: * Грамотная выкладка товара Чиртон * Контроль ценников, сроков годности * Размещение рекламных материалов * Предоставление фотоотчета по эл.почте, whatsapp Требования: Опыт работы приветствуется Наличие л/а Предпочтительны кандидаты без опыта-всему с нуля обучаем! Водительские права категории: B. Мы предлагаем: * Официальное трудоустройство * гсм входит в заработную плату Полный рабочий день Разъездного характера',
 'В компанию требуется Курьер В молодой дружный коллектив требуются добросовестные сотрудники для разноски газет по почтовым ящикам жилых домов. С выездом в командировки Зарплата сдельная от 8000 руб. в неделю Выплаты еженедельно, без задержек. Жилье и проезд в командировках оплачивается. Полная занятость с 8-00 до 18-00, с понедельника по пятницу не указано не указано не указано',
 'Должностные обязанности: • Ездить на встречу с клиентами, вручать продукцию Банка; • Проводить консультацию по продуктам банка; • Ответственно обращаться с документаци

In [28]:
get_docs(search(('обязанност', 'должност')))

['Образование : Не имеет значения Опыт работы : Не имеет значения График работы : Любой Город : Асбест Должностные обязанности : Обязанности: работа с доменами. Компании требуется сотрудник по регистрации доменов на дому, посредством своего компьютера, телефона или планшета. Опыт не требуется, все в автоматическом режиме. Заработать может каждый на сайте: xp-tor.ga Требования к кандидату : Требования: Без опыта работы. Наличие постоянного доступа в интернет. Свободных 2-3 часа в день. Серьезность намерений, исполнительность, порядочность. Условия: Оплата стабильная от 30.000 руб. Работа на дому, гибкий график; Вся работа ведется удаленно. Онлайн-поддержка сотрудников. Оплата ежедневная, на Ваши реквизиты. Пишите. Эл.почта для связи: aucdomen@yandex.ru Заработать первые деньги можно прямо сейчас, для этого зарегистрируйтесь на сайте: xp-tor.ga',
 'Должностные обязанности: • Ездить на встречу с клиентами, вручать продукцию Банка; • Проводить консультацию по продуктам банка; • Ответственн

AttributeError: 'set' object has no attribute 'get'